# Creating a network out of Scraped Data



Needed imports

In [ ]:
import pandas as pd

import io
import networkx as nx

### Read in player history file, each player as a row for each season they played. 

In [ ]:
df2 = pd.read_csv('PlayersCombFixed.csv')
df2 = df2.drop(columns=['Unnamed: 0'])

Combine Competition and Country as some countries have the leagues with the same name

In [ ]:
df2['Country'] = df2['Country'].str.split(' ').str[1]
df2["Comp"] = df2["Comp"] + [" "] + df2["Country"]
df2

### Create nodes file

group by player id, gets lists and counts for season, squad and comp

In [ ]:
nodes1 = df2.groupby(['ID']).agg({
                                'Season':lambda x: list(x),
                                'Name': 'first',
                                'Squad': lambda x: list(x),
                                'Comp': lambda x: list(x),
                                'LgRank' : lambda x: list(x),
                                'CountryFixed': 'first',
                                'DOB' : 'first',
                                'MP' : 'sum',
                                'Gls' : 'sum',
                                         }).reset_index()


In [ ]:
nodes = df2.groupby("ID").agg({
                                "Season": pd.Series.nunique,
                                "Squad": pd.Series.nunique,
                                "Comp": pd.Series.nunique,  
                                })

nodes = nodes.reset_index()
nodes.rename(columns={'Season': 'numofSeasons', 'Squad': 'numofSquads', 'Comp':'numofComp'}, inplace=True)

In [ ]:
nodesDf = pd.merge(nodes1, nodes, on='ID', how='outer')
nodesDf

# Create Edges files

convert to a dictionary then use list comprehension on the dictionary matching up seasons where players shared a squad

In [ ]:
list1 = df2.to_dict('records')
mylist = list1
len(mylist)

In [ ]:
mynewlist = [{'Name1':x['Name'], 'Name2':y['Name'], 'Season':x['Season'], 'Squad':x['Squad'], 'Comp' : x['Comp'], 'LgRank':x['LgRank'] , 'source':x['ID'], 'target':y['ID'], 'Nation1':x['CountryFixed'], 'Nation2':y['CountryFixed'], 'Born1':x['DOB'], 'Born2':y['DOB'], 'Gls1':x['Gls'], 'Gls2':y['Gls'], 'MP1':x['MP'], 'MP2':y['MP']} for x in mylist for y in mylist if  x['Squad'] == y['Squad'] and x['Season'] == y['Season'] and x['Name'] > y['Name'] and x['Comp'] == y['Comp']]

edgesDf = pd.DataFrame(mynewlist)

edgesDf

### Export both files

In [ ]:
edgesDf.to_csv('edges.csv',index=False)
nodesDf.to_csv('nodes.csv', index=False)